In [ ]:
# %pip install kagglehub
import kagglehub
import shutil # 파일 옮겨주는 라이브러리

# 데이터 다운로드 
## 파일이 저장된 path 
data_path = kagglehub.dataset_download("thedevastator/2010-new-york-state-hospital-inpatient-discharge")
target_path = 'datasets'
shutil.move(data_path, target_path)

'datasets\\1'

In [2]:
import pandas as pd
import numpy as np

# 데이터 불러오기 
data = pd.read_csv('datasets/hospital-inpatient-discharges-sparcs-de-identified-2010-1.csv')
data.head(10)

C:\Users\jinhy\AppData\Local\Temp\ipykernel_6756\1662323173.py:5: DtypeWarning: Columns (1,2,7,11,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('datasets/hospital-inpatient-discharges-sparcs-de-identified-2010-1.csv')


,index,Health Service Area,Hospital County,Operating Certificate Number,Facility ID,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,...,Source of Payment 2,Source of Payment 3,Attending Provider License Number,Operating Provider License Number,Other Provider License Number,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs
0,0,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,18 to 29,NaN,F,White,...,Self-Pay,NaN,3623.0,NaN,NaN,0,N,Y,4476.23,1672.65
1,1,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,50 to 69,NaN,M,White,...,Medicare,Blue Cross,216951.0,216951.0,NaN,0,N,Y,148612.34,51414.70
2,2,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,70 or Older,NaN,M,White,...,Blue Cross,Self-Pay,3076.0,NaN,NaN,0,N,N,16561.99,4032.49
3,3,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,70 or Older,NaN,F,White,...,Insurance Company,Self-Pay,140796.0,140796.0,170254.0,0,N,N,13593.51,4947.81
4,4,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,70 or Older,NaN,F,White,...,Insurance Company,Self-Pay,170254.0,170254.0,170254.0,0,N,N,31962.58,16988.33
5,5,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,100.0,M,White,...,Self-Pay,NaN,218155.0,NaN,NaN,0,N,N,8420.75,3246.34
6,6,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,100.0,M,White,...,Self-Pay,NaN,207219.0,207219.0,NaN,0,N,Y,21090.63,6194.34
7,7,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,100.0,F,White,...,Self-Pay,NaN,244921.0,NaN,NaN,0,N,Y,3571.75,1435.34
8,8,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,0 to 17,100.0,F,White,...,Self-Pay,NaN,242288.0,NaN,NaN,0,N,Y,7850.84,2278.30
9,9,Capital/Adiron,Albany,101000.0,1.0,Albany Medical Center Hospital,30 to 49,100.0,M,White,...,Self-Pay,NaN,191936.0,191936.0,NaN,0,N,N,13787.00,7491.89


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

# data.columns
X_columns = data.drop(columns='Length of Stay').columns
cate_columns = ['Health Service Area', 'Hospital County', 'Facility Name', 'Age Group', 'Gender', 'Race', 'Ethnicity', 'Type of Admission', 'Patient Disposition', 'CCS Diagnosis Description', 'CCS Procedure Description', 'APR DRG Description', 'APR MDC Description', 'APR Medical Surgical Description', 'Source of Payment 1']
num_columns = ['Operating Certificate Number', 'Facility ID', 'Discharge Year', 'CCS Diagnosis Code', 'CCS Procedure Code', 'APR DRG Code', 'APR MDC Code', 'APR Severity of Illness Code']


preprocessor = ColumnTransformer([
    ('imp', SimpleImputer(strategy='most_frequent', missing_values=np.nan), cate_columns),
    # ('imp2', SimpleImputer(strategy='most_frequent', missing_values='Minor'), cate_columns),
    ('ohe', OneHotEncoder(), cate_columns),
    ('imp3', SimpleImputer(strategy='most_frequent', missing_values=np.nan), num_columns),
    # ('imp4', SimpleImputer(strategy='constant', missing_values='Minor', fill_value=0.0), num_columns),    
    ('std', StandardScaler(), num_columns)
])

# params = (
#     'n_estimators' : 500,
#     'max_depth' : 5
# )

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfr', RandomForestRegressor(n_estimators=500, max_depth=5))
])

In [9]:
X = data.drop(columns='Length of Stay')
y = data['Length of Stay'].to_frame()
X.shape, y.shape
data.isna().sum() # Health Service Area / Hospital County / Operating Certificate Number / Facility ID / Patient Disposition / CCS * 4 / APR Severity of Illness Description / APR Risk of Martality
# da = data['Patient Disposition']
# for column in data.columns:
#     da = data[column]
#     if any(da=='Minor'):
#         print(f'{column}')
#     else:
#         print(data[da == 'Minor'].sum().sum(), f'{column}')
# imp = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
# result = imp.fit_transform(da.to_frame())
# res = pd.DataFrame(result)
# res.value_counts().sort_values(axis=0)


index                                        0
Health Service Area                       4887
Hospital County                           4887
Operating Certificate Number              4887
Facility ID                               4887
Facility Name                                0
Age Group                                    0
Zip Code - 3 digits                       3201
Gender                                       0
Race                                         0
Ethnicity                                    0
Length of Stay                               0
Type of Admission                            0
Patient Disposition                        103
Discharge Year                               0
CCS Diagnosis Code                        2273
CCS Diagnosis Description                 2273
CCS Procedure Code                        2273
CCS Procedure Description                 2273
APR DRG Code                                 0
APR DRG Description                          0
APR MDC Code 

In [10]:
pred = pipeline.fit(X)

TypeError: LabelEncoder.fit_transform() takes 2 positional arguments but 3 were given

In [167]:
data.columns

Index(['index', 'Health Service Area', 'Hospital County',
       'Operating Certificate Number', 'Facility ID', 'Facility Name',
       'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race', 'Ethnicity',
       'Length of Stay', 'Type of Admission', 'Patient Disposition',
       'Discharge Year', 'CCS Diagnosis Code', 'CCS Diagnosis Description',
       'CCS Procedure Code', 'CCS Procedure Description', 'APR DRG Code',
       'APR DRG Description', 'APR MDC Code', 'APR MDC Description',
       'APR Severity of Illness Code', 'APR Severity of Illness Description',
       'APR Risk of Mortality', 'APR Medical Surgical Description',
       'Source of Payment 1', 'Source of Payment 2', 'Source of Payment 3',
       'Attending Provider License Number',
       'Operating Provider License Number', 'Other Provider License Number',
       'Birth Weight', 'Abortion Edit Indicator',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs'],
      dtype='object')

In [ ]:
'''
, 'Source of Payment 2', 'Source of Payment 3'
'Abortion Edit Indicator', 'Emergency Department Indicator', 'Zip Code - 3 digits'
, 'Attending Provider License Number', 'Operating Provider License Number', 'Other Provider License Number', 'Birth Weight', 'Total Charges', 'Total Costs'
'''